In [ ]:
%%writefile 5.cu

#include <stdio.h>
#include <cuda_runtime.h>

#define N 5

__device__ int d_c[N];


__global__ void AddArray(int *d_a, int *d_b){
    int i = threadIdx.x;

    if (i < N){
        d_c[i]= d_a[i] + d_b[i];
    }
}


int main(){
  int h_a[N] = {1,2,3,4,5};
  int h_b[N] = {1,2,3,4,5};
  int h_c[N];

  int *d_a, *d_b;

  cudaMalloc((void **)&d_a, N * sizeof(int));
  cudaMalloc((void **)&d_b, N * sizeof(int));

  cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);

  AddArray<<<1, N>>> (d_a, d_b);
  cudaDeviceSynchronize();

  cudaMemcpyFromSymbol(h_c, d_c, N * sizeof(int), 0, cudaMemcpyDeviceToHost);


  for(int i=0; i<N; i++){
    printf("%d ", h_c[i]);

  }
  printf("\n");

  cudaFree(d_a);
  cudaFree(d_b);

  return 0;
}


Overwriting 5.cu


In [ ]:
!nvcc -o 5 5.cu

In [ ]:
!./5

2 4 6 8 10 


In [ ]:
%%writefile 6.cu

#include<stdio.h>
#include<cuda_runtime.h>

__global__ void div(){
  int i = threadIdx.x;
  if(i%2==0){
    printf("thread %d HPCAP\n",i);
  }
  else{
    printf("thread %d CUDA\n",i);
  }
}

int main(){
  int n=5;

  div<<<1,n>>>();
  cudaDeviceSynchronize();

  return 0;
}


Overwriting 6.cu


In [ ]:
!nvcc -o 6 6.cu

In [ ]:
!./6

thread 1 CUDA
thread 3 CUDA
thread 0 HPCAP
thread 2 HPCAP
thread 4 HPCAP


In [ ]:
%%writefile 7.cu

#include<stdio.h>
#include<cuda_runtime.h>

__global__ void div(){
  int i = threadIdx.x;
  if(i%2==0){
    printf("thread %d is even \n",i);
  }
  else{
    printf("thread %d is odd\n",i);
  }
}

int main(){
  int n=99;

  div<<<1,n>>>();
  cudaDeviceSynchronize();

  return 0;
}

Writing 7.cu


In [ ]:
!nvcc -o 7 7.cu

In [ ]:
!./7

thread 97 is odd
thread 1 is odd
thread 3 is odd
thread 5 is odd
thread 7 is odd
thread 9 is odd
thread 11 is odd
thread 13 is odd
thread 15 is odd
thread 17 is odd
thread 19 is odd
thread 21 is odd
thread 23 is odd
thread 25 is odd
thread 27 is odd
thread 29 is odd
thread 31 is odd
thread 33 is odd
thread 35 is odd
thread 37 is odd
thread 39 is odd
thread 41 is odd
thread 43 is odd
thread 45 is odd
thread 47 is odd
thread 49 is odd
thread 51 is odd
thread 53 is odd
thread 55 is odd
thread 57 is odd
thread 59 is odd
thread 61 is odd
thread 63 is odd
thread 65 is odd
thread 67 is odd
thread 69 is odd
thread 71 is odd
thread 73 is odd
thread 75 is odd
thread 77 is odd
thread 79 is odd
thread 81 is odd
thread 83 is odd
thread 85 is odd
thread 87 is odd
thread 89 is odd
thread 91 is odd
thread 93 is odd
thread 95 is odd
thread 96 is even 
thread 98 is even 
thread 0 is even 
thread 2 is even 
thread 4 is even 
thread 6 is even 
thread 8 is even 
thread 10 is even 
thread 12 is even 
thread

In [ ]:

%%writefile 8.cu

#include<stdio.h>
#include<cuda_runtime.h>

__global__ void div(){

  int i = blockDim.x*blockIdx.x+threadIdx.x;
  if(i>=0){
    int count=0;
    for(int j=1;j<=i;j++){
      if(i%j==0){
        count++;
      }
    }
    if(count==2){
      printf("thread %d is prime \n",i);
    }
    else{
      printf("thread %d is not prime \n",i);
    }
  }
}

int main(){
  int n=1000;

  div<<<11,n>>>();
  cudaDeviceSynchronize();

  return 0;
}

Overwriting 8.cu


In [ ]:
!nvcc 8.cu -o 8

In [ ]:
!./8

thread 5324 is not prime 
thread 5325 is not prime 
thread 5326 is not prime 
thread 5327 is not prime 
thread 5328 is not prime 
thread 5329 is not prime 
thread 5330 is not prime 
thread 5331 is not prime 
thread 5332 is not prime 
thread 5334 is not prime 
thread 5335 is not prime 
thread 5336 is not prime 
thread 5337 is not prime 
thread 5338 is not prime 
thread 5339 is not prime 
thread 5340 is not prime 
thread 5341 is not prime 
thread 5342 is not prime 
thread 5343 is not prime 
thread 5344 is not prime 
thread 5345 is not prime 
thread 5346 is not prime 
thread 5348 is not prime 
thread 5349 is not prime 
thread 5350 is not prime 
thread 8039 is prime 
thread 8053 is prime 
thread 8059 is prime 
thread 8231 is prime 
thread 8233 is prime 
thread 8237 is prime 
thread 8243 is prime 
thread 5160 is not prime 
thread 5161 is not prime 
thread 5162 is not prime 
thread 5163 is not prime 
thread 5164 is not prime 
thread 5165 is not prime 
thread 5166 is not prime 
thread 5168 is

In [ ]:
%%writefile 9.cu

#include <stdio.h>

__constant__ float lookupTable[256];

__global__ void computeWithLookupTable(float *output, int n){
  int id = threadIdx.x + blockIdx.x * blockDim.x;

  if (id < n){
    output[id] = lookupTable[id%256]*2.0f;
  }
}

int main(){

  const int arraySize = 1024;
  float h_output[arraySize];
  float *d_output;

  cudaMalloc(&d_output, arraySize * sizeof(float));


  int threadPerBlock = 256;
  int blockPerGrid = (arraySize + threadPerBlock - 1) / threadsPerBlock;
  computeWithLookupTable<<<blocksPerGrid, threadsPerBlock>>> (d_output,arraySize );

  cudaMemcpy(h_output, d_output, arraySize * sizeof(float), cudaMemcpyDeviceToHost);

  for(int i=0; i<arraySize; i++){
    printf("%f ", h_output[i]);
  }
  printf("\n");

  cudaFree(d_output);
  return 0;
}

Writing 9.cu


In [ ]:
%%writefile 11.cu

#include<stdio.h>

__global__ void count(int *deviceData,int *deviceResult,int size){
  int i = blockDim.x*blockIdx.x+threadIdx.x;
  if(i<size){
    if(deviceData[i]>50){
      deviceResult[i]=1;
    }else{
      deviceResult[i]=0;
    }
  }
}
int main(){
  const int N=100;

  int hostData[N];
  int hostResult[N];

  int *deviceData;
  int *deviceResult;

  cudaMalloc(&deviceData,N*sizeof(int));
  cudaMalloc(&deviceResult,N*sizeof(int));

  for(int i=0;i<N;i++){
    hostData[i]=i+1;
  }

  cudaMemcpy(deviceData,hostData,N*sizeof(int),cudaMemcpyHostToDevice);

  count<<<1,N>>>(deviceData,deviceResult,N);

  cudaMemcpy(hostResult,deviceResult,N*sizeof(int),cudaMemcpyDeviceToHost);

  int sum=0;
  for(int i=0;i<N;i++){
    sum += hostResult[i];
  }
  printf("sum is %d\n",sum);
  cudaFree(deviceData);
  cudaFree(deviceResult);

  return 0;
}

Overwriting 11.cu


In [ ]:
!nvcc -o 11 11.cu

In [ ]:
!./11

sum is 50


In [ ]:
%%writefile 12.cu

#include<stdio.h>
#include<cuda_runtime.h>

__global__ void div(int *result,int n){
  int i = threadIdx.x;
  if(i<n){

  if(i%2==0){
    result[i]=1;
  }
  else{
    result[i]=0;
  }
  }
}

int main(){
  int n=99;
  int *hostResult;
  hostResult = (int *)malloc(n*sizeof(int));

  int *result;
  cudaMalloc(&result,n*sizeof(int));

  div<<<1,n>>>(result,n);
  cudaDeviceSynchronize();

  cudaMemcpy(hostResult,result,n*sizeof(int),cudaMemcpyDeviceToHost);

  int evenCount=0;
  int oddCount=0;
  for(int i=0;i<n;i++){
    if(hostResult[i]==1){
      evenCount++;
    }
    else{
      oddCount++;
    }
  }
  printf("even count is %d\n",evenCount);
  printf("odd count is %d\n",oddCount);

  cudaFree(result);
  free(hostResult);
  return 0;
}

Writing 12.cu


In [ ]:
!nvcc -o 12 12.cu

In [ ]:
!./12

even count is 50
odd count is 49


In [ ]:
%%writefile 13.cu

#include<stdio.h>
#include<cuda_runtime.h>

__global__ void multiplesOfSeven(int *deviceData,int *deviceResult,int size){
  int i = blockDim.x * blockIdx.x + threadIdx.x;

  if(i<size){
    if(deviceData[i]%7==0){
      deviceResult[i]=1;
    }
  }
}

int main(){
  const int N = 1000;

  int *hostData;
  int *hostResult;

  hostData = (int *)malloc(N*sizeof(int));
  hostResult = (int *)malloc(N*sizeof(int));

  for(int i=0;i<N;i++){
    hostData[i]=i+1;
  }

  int *deviceData;
  int *deviceResult;

  cudaMalloc(&deviceData,N*sizeof(int));
  cudaMalloc(&deviceResult,N*sizeof(int));

  cudaMemcpy(deviceData,hostData,N*sizeof(int),cudaMemcpyHostToDevice);

  multiplesOfSeven<<<1,N>>>(deviceData,deviceResult,N);

  cudaMemcpy(hostResult,deviceResult,N*sizeof(int),cudaMemcpyDeviceToHost);
  int sum=0;
  for(int i=0;i<N;i++){
    sum += hostResult[i];
  }
  printf("Total no of numbers that are divisible by 7 from 0 to %d are %d\n",N,sum);

}

Overwriting 13.cu


In [ ]:
!nvcc -o 13 13.cu

In [ ]:
!./13

Total no of numbers that are divisible by 7 from 0 to 1000 are 142


In [ ]:
%%writefile 14.cu

#include<stdio.h>
#include<cuda_runtime.h>

__global__ void rightCircularShift(int *deviceData,int *deviceResult,int size){
  int i = blockDim.x * blockIdx.x + threadIdx.x;

  if(i<size){
    deviceResult[i]= deviceData[size-1]-i;
  }
}

int main(){
  const int N = 10;

  int *hostData;
  int *hostResult;

  hostData = (int *)malloc(N*sizeof(int));
  hostResult = (int *)malloc(N*sizeof(int));

  for(int i=0;i<N;i++){
    hostData[i]=i;
  }

  int *deviceData;
  int *deviceResult;

  cudaMalloc(&deviceData,N*sizeof(int));
  cudaMalloc(&deviceResult,N*sizeof(int));

  cudaMemcpy(deviceData,hostData,N*sizeof(int),cudaMemcpyHostToDevice);

  rightCircularShift<<<1,N>>>(deviceData,deviceResult,N);

  cudaMemcpy(hostResult,deviceResult,N*sizeof(int),cudaMemcpyDeviceToHost);

  printf("Original Array: ");
  for(int i=0;i<N;i++){
    printf("%d ",hostData[i]);
  }
  printf("\n");

  printf("ResultantArray: ");
  for(int i=0;i<N;i++){
    printf("%d ",hostResult[i]);
  }
  printf("\n");


  return 0;
}

Overwriting 14.cu


In [ ]:
!nvcc -o 14 14.cu

In [ ]:
!./14

Original Array: 0 1 2 3 4 5 6 7 8 9 
ResultantArray: 9 8 7 6 5 4 3 2 1 0 
